<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>IA Generativa e LLMs Para Processamento de Linguagem Natural</font>
## <font color='blue'>Projeto 8 - Treinamento</font>
## <font color='blue'>Ajuste Fino com Nossos Próprios Dados e Deploy de LLM via App Web</font>

## Instalando e Carregando os Pacotes

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [2]:
%env TF_CPP_MIN_LOG_LEVEL=3

env: TF_CPP_MIN_LOG_LEVEL=3


In [3]:
!pip install -q torch

In [4]:
!pip install -q transformers

In [5]:
!pip install -q accelerate

In [6]:
# Imports
import torch
import accelerate
import transformers
import numpy as np
import torch.nn.functional as F
from accelerate import Accelerator
from torch.utils.data import Dataset
from transformers import AutoConfig, AutoModelForCausalLM, GPT2Tokenizer
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Carregando o LLM

https://huggingface.co/gpt2

O modelo terá a mesma arquitetura do GPT-2, mas com algumas modificações para torná-lo menor. 

As principais mudanças são: 

- O tamanho do vocabulário que será 13 (porque só vai lidar com números), mais o padding token, o "+" e o "=". 

- A janela de contexto suportará apenas 6 tokens, pois estamos interessados apenas em realizar a adição de dois dígitos únicos (de fato, prever o dígito de saída).

O objetivo aqui é demonstrar como realmente funciona o processo de preparação dos dados para um LLM.
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
Nota: LLMs não fazem Matemática (pelo menos não ainda). LLMs fazem a previsão da próxima palavra ou caracter. Fazer com que os LLMs sejam capazes de realizar operações matemáticas (bem como outras tarefas cognitivas) é uma área ativa de pesquisa chamada Reasoning. 

Colocamos uma descrição completa do conceito de Reasoning e a indicação de um paper de pesquisa no e-book disponível no Capítulo 17 do curso.

In [8]:
# Tamanho do vocabulário
vocab_size = 13

In [9]:
# Comprimento da sequência
sequence_length = 4

In [10]:
# Comprimento do resultado
result_length = 2

In [11]:
# Comprimento do contexto
context_length = sequence_length + result_length

In [12]:
# Parâmetros de configuração do modelo GPT-2
config = AutoConfig.from_pretrained("gpt2", 
                                    vocab_size = vocab_size, 
                                    n_ctx = context_length, 
                                    n_head = 4, 
                                    n_layer = 2) 

In [13]:
# Carrega o modelo
modelo = AutoModelForCausalLM.from_config(config)

In [14]:
# Função para calcular o tamanho do modelo
def dsa_calcula_tamanho_modelo(model):
    return sum(t.numel() for t in model.parameters())

In [15]:
print(f'Tamanho do Modelo: {dsa_calcula_tamanho_modelo(modelo) / 1000 ** 2:.1f}M parâmetros')

Tamanho do Modelo: 15.0M parâmetros


Este modelo tem 15 milhões de parâmetros em vez dos 111 milhões de parâmetros da configuração padrão "gpt2".

In [16]:
type(modelo)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

## Criando Tokenizador Personalizado

In [17]:
# Definindo uma classe chamada DSATokenizer, que é usada para tokenizar os números
class DSATokenizer:
    
    # Método construtor da classe, que é executado quando um objeto dessa classe é criado
    def __init__(self, numbers_qty = 10):
        
        # Lista de tokens possíveis que o tokenizador pode encontrar
        vocab = ['+', '=', '-1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
        
        # Definindo a quantidade de números que o tokenizador pode lidar
        self.numbers_qty = numbers_qty
        
        # Definindo o token de preenchimento (padding)
        self.pad_token = '-1'
        
        # Criando um dicionário que mapeia cada token para um índice único
        self.encoder = {str(v):i for i,v in enumerate(vocab)}
        
        # Criando um dicionário que mapeia cada índice único de volta ao token correspondente
        self.decoder = {i:str(v) for i,v in enumerate(vocab)}
        
        # Obtendo o índice do token de preenchimento no encoder
        self.pad_token_id = self.encoder[self.pad_token]

    # Método para decodificar uma lista de IDs de token de volta para uma string
    def decode(self, token_ids):
        return ' '.join(self.decoder[t] for t in token_ids)

    # Método que é chamado quando o objeto da classe é invocado como uma função
    def __call__(self, text):
        
        # Dividindo o texto em tokens individuais e retornando uma lista dos IDs correspondentes
        return [self.encoder[t] for t in text.split()]

> Vamos testar o tokenizador!

In [18]:
# Cria o objeto do tokenizador
tokenizer = DSATokenizer(vocab_size)

In [19]:
# Decoder do tokenizador
tokenizer.decoder

{0: '+',
 1: '=',
 2: '-1',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

Queremos que o modelo aprenda a gerar o número 2 toda vez que ele receber como entrada: 1 + 1 =

E para isso precisamos aplicar a tokenização com base no vocabulário.

In [20]:
# Testando o tokenizador
tokenizer("1 + 1 = 2")

[4, 0, 4, 1, 5]

In [21]:
# Testando o tokenizador (por que o erro?)
tokenizer("9 + 5 = 14")

KeyError: '14'

In [22]:
# Testando o tokenizador
tokenizer("9 + 5 = 1 4")

[12, 0, 8, 1, 4, 7]

In [23]:
# Testando o tokenizador (por que o erro?)
tokenizer("19 + 5 = 2 4")

KeyError: '19'

## Organizando o Formato dos Dados

O conjunto de dados deve ser criado neste formato:

- Entrada: "2 + 3 = 0" onde os 4 primeiros caracteres representam a sequência de entrada e o quinto caractere representa o primeiro caracter da saída.
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
- Saída: "+ 3 = 0 5" onde os 2 últimos dígitos representam o resultado da adição e os 3 primeiros dígitos são ignorados durante o treinamento e preenchidos com o pad.

O resultado é um conjunto de dados de sequências tokenizadas de números.

In [24]:
# Definindo uma classe chamada CriaDataset, que herda da classe Dataset do PyTorch
class DSAOrganizaDataset(Dataset):

    # Método construtor da classe, que é executado quando um objeto dessa classe é criado
    def __init__(self, split, length = 6):
        
        # Verificando se a divisão do dataset (split) é 'treino' ou 'teste'
        assert split in {'treino', 'teste'}
        self.split = split
        self.length = length
    
    # Definindo o método len que retorna o tamanho do dataset. 
    # Nesse caso, o tamanho é fixo e igual a 1 milhão.
    def __len__(self):
        return 1000000 

    # Definindo o método getitem que é usado para obter um item específico do dataset
    def __getitem__(self, idx):

        # Criando uma lista com todos os números disponíveis que não são tokens de padding e são numéricos
        available_numbers = [int(n) for n in tokenizer.decoder.values() if n != tokenizer.pad_token and str(n).isnumeric()]
        
        # Selecionando aleatoriamente números da lista de números disponíveis para criar uma entrada (input)
        inp = torch.tensor(np.random.choice(available_numbers, size = result_length))
        
        # Calculando a soma dos números selecionados e criando um tensor
        sol = torch.tensor([int(i) for i in str(inp.sum().item())])
        
        # Preenchendo o tensor com zeros para que tenha o tamanho desejado
        sol = torch.nn.functional.pad(sol, (1 if sol.size()[0] == 1 else 0,0), 'constant', 0)

        # Concatenando a entrada e a solução em um tensor
        cat = torch.cat((inp, sol), dim = 0)

        # Criando os tensores de entrada e alvo para o treinamento do modelo
        x = cat[:-1].clone()
        y = cat[1:].clone()

        # Definindo o primeiro elemento do tensor alvo como o token de padding
        y[:1] = int(tokenizer.pad_token)

        # Transformando os tensores x e y em strings
        x = str(x[0].item()) + ' + ' + str(x[1].item()) + ' = ' + str(x[2].item())
        y = '-1 ' + str(y[0].item()) + ' -1 ' + str(y[1].item()) + ' ' + str(y[2].item())
        
        # Tokenizando as strings de entrada e alvo
        tokenized_input = tokenizer(x)
        tokenized_output = tokenizer(y)
        
        # Retornando os tensores de entrada e alvo como itens do dataset
        return torch.tensor(tokenized_input), torch.tensor(tokenized_output)

## Criando os Datasets de Treino e Teste

In [25]:
# Prepara o dataset de treino
dataset_treino = DSAOrganizaDataset('treino', length = sequence_length)

In [26]:
# Prepara o dataset de teste
dataset_teste = DSAOrganizaDataset('teste', length = sequence_length)

In [27]:
x, y = dataset_treino[0]

In [28]:
x

tensor([12,  0,  9,  1,  4])

In [29]:
y

tensor([2, 2, 2, 4, 8])

In [30]:
print(tokenizer.decode(x.numpy()))

9 + 6 = 1


In [31]:
print(tokenizer.decode(y.numpy()))

-1 -1 -1 1 5


## Criando o Loop de Treinamento

In [32]:
# Hiperparâmetros
num_epochs = 2
batch_size = 100

In [33]:
# Otimizador
optimizer = torch.optim.Adam(modelo.parameters())

In [34]:
# Dataloader
dados = torch.utils.data.DataLoader(dataset_treino, shuffle = True, batch_size = batch_size)

https://pypi.org/project/accelerate/

In [35]:
# Cria o acelerador
accelerator = Accelerator()

In [36]:
# Otimiza modelo, otimizador e dados para o dispositivo
modelo, optimizer, dados = accelerator.prepare(modelo, optimizer, dados)

In [37]:
# Arquitetura do modelo que será treinado
modelo.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(13, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-1): 2 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=13, bias=False)
)

## Treinando o LLM (Ajuste Fino) com Nossos Próprios Dados

In [38]:
%%time

print('\nIniciando o Ajuste Fino do LLM... Seja Paciente e Aguarde!')

# Iniciando o loop para as épocas de treinamento
for epoch in range(num_epochs):

    # Iterando por cada batch (conjunto) de dados de entrada e alvos no dataset de treinamento
    for source, targets in dados:

        # Resetando os gradientes acumulados no otimizador
        optimizer.zero_grad()

        # Calculando a perda (loss) através da entropia cruzada 
        # entre as previsões do modelo e os alvos verdadeiros. 
        # Os tensores são "achatados" para que possam ser passados para a função de entropia cruzada. 
        # O índice do token de preenchimento (pad_token) é ignorado no cálculo da perda.
        loss = F.cross_entropy(modelo(source).logits.flatten(end_dim = 1), 
                               targets.flatten(end_dim = 1), 
                               ignore_index = tokenizer.pad_token_id)

        # Calculando os gradientes da perda em relação aos parâmetros do modelo
        accelerator.backward(loss)

        # Atualizando os parâmetros do modelo utilizando os gradientes calculados
        optimizer.step()

        # Recalculando a perda após a etapa de otimização. 
        loss = F.cross_entropy(modelo(source).logits.flatten(end_dim = 1), 
                               targets.flatten(end_dim = 1), 
                               ignore_index = tokenizer.pad_token_id)

    # Imprimindo a época atual e a perda após cada época de treinamento
    print(f'\nEpoch: {epoch+1} / {num_epochs} --- Erro: {loss.item()}')

print('\nAjuste Fino do LLM Concluído com Sucesso.\n')


Iniciando o Ajuste Fino do LLM... Seja Paciente e Aguarde!

Epoch: 1 / 2 --- Erro: 0.13560616970062256

Epoch: 2 / 2 --- Erro: 0.023130610585212708

Ajuste Fino do LLM Concluído com Sucesso.

CPU times: user 4min 53s, sys: 44.6 s, total: 5min 38s
Wall time: 6min 22s


## Avaliação do LLM

In [39]:
# Definindo a função gera_solution com três parâmetros: input, solution_length e model
def dsa_faz_previsao(entrada, solution_length = 6, model = modelo):

    # Colocando o modelo em modo de avaliação. 
    model.eval()

    # Convertendo a entrada (string) em tensor utilizando o tokenizer. 
    # O tensor é uma estrutura de dados que o modelo de aprendizado de máquina pode processar.
    entrada = torch.tensor(tokenizer(entrada))

    # Enviando o tensor de entrada para o dispositivo de cálculo disponível (CPU ou GPU)
    entrada = entrada.to(accelerator.device)

    # Iniciando uma lista vazia para armazenar a solução
    solution = []

    # Loop que gera a solução de comprimento solution_length
    for i in range(solution_length):

        # Alimentando a entrada atual ao modelo e obtendo a saída
        saida = model(entrada)

        # Pegando o índice do maior valor no último conjunto de logits (log-odds) da saída, 
        # que é a previsão do modelo para o próximo token
        predicted = saida.logits[-1].argmax()

        # Concatenando a previsão atual com a entrada atual. 
        # Isso servirá como a nova entrada para a próxima iteração.
        entrada = torch.cat((entrada, predicted.unsqueeze(0)), dim = 0)

        # Adicionando a previsão atual à lista de soluções e convertendo o tensor em um número Python padrão
        solution.append(predicted.cpu().item())

    # Decodificando a lista de soluções para obter a string de saída e retornando-a
    return tokenizer.decode(solution)

In [40]:
# Definindo a função avalia_modelo com dois parâmetros: num_samples e log
def dsa_avalia_modelo(num_samples = 1000, log = False):

    # Iniciando um contador para as previsões corretas
    correct = 0

    # Loop que itera num_samples vezes
    for i in range(num_samples):

        # Obtendo a entrada e o alvo (resposta correta) do i-ésimo exemplo do conjunto de teste
        entrada, target = dataset_teste[i]

        # Convertendo os tensores de entrada e alvo em arrays numpy para processamento posterior
        entrada = entrada.cpu().numpy()
        target = target.cpu().numpy()

        # Decodificando a entrada e o alvo utilizando o tokenizer
        entrada = tokenizer.decode(entrada[:sequence_length])
        target = tokenizer.decode(target[sequence_length-1:])

        # Gerando a previsão utilizando a função faz_previsao
        predicted = dsa_faz_previsao(entrada, solution_length = result_length, model = modelo)
 
        # Se a previsão for igual ao alvo, incrementa o contador de previsões corretas
        if target == predicted:
            correct += 1
            # Se log for True, imprime detalhes do exemplo e a previsão correta
            if log:
                print(f'Acerto do Modelo: Input: {entrada} Target: {target} Previsão: {predicted}')
        else:
            # Se log for True, imprime detalhes do exemplo e a previsão errada
            if log:
                print(f'Erro do Modelo: Input: {entrada} Target: {target} Previsão: {predicted}')

    # Ao final do loop, calcula a acurácia (número de previsões corretas dividido pelo número total de exemplos) 
    print(f'\nAcurácia: {correct/num_samples}')

In [41]:
# Executa a função
dsa_avalia_modelo(num_samples = 10, log = True)

Acerto do Modelo: Input: 7 + 2 = Target: 0 9 Previsão: 0 9
Acerto do Modelo: Input: 4 + 6 = Target: 1 0 Previsão: 1 0
Acerto do Modelo: Input: 5 + 1 = Target: 0 6 Previsão: 0 6
Acerto do Modelo: Input: 0 + 5 = Target: 0 5 Previsão: 0 5
Acerto do Modelo: Input: 8 + 7 = Target: 1 5 Previsão: 1 5
Acerto do Modelo: Input: 6 + 4 = Target: 1 0 Previsão: 1 0
Acerto do Modelo: Input: 0 + 2 = Target: 0 2 Previsão: 0 2
Acerto do Modelo: Input: 1 + 5 = Target: 0 6 Previsão: 0 6
Acerto do Modelo: Input: 9 + 7 = Target: 1 6 Previsão: 1 6
Acerto do Modelo: Input: 5 + 1 = Target: 0 6 Previsão: 0 6

Acurácia: 1.0


In [42]:
# Executa a função
dsa_avalia_modelo(num_samples = 1000, log = False)


Acurácia: 1.0


In [43]:
type(modelo)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [44]:
modelo.save_pretrained("llm/llm_dsa_final")

In [45]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [46]:
#%watermark -v -m

In [47]:
#%watermark --iversions

# Fim